## Parts of this notebook are borrowed from [this repo](https://github.com/adithya-s-k/AI-Engineering.academy)



In [1]:
# !pip3 install -q -U fsspec==2024.6.1
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
# !pip3 install -q -U transformers==4.39.1
!pip3 install -q -U transformers==4.38.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [3]:
# !pip uninstall bitsandbytes accelerate -y
# !pip install -i https://pypi.org/simple/ bitsandbytes

In [4]:
# Get HF token, disable wandb
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["WANDB_DISABLED"] = "true"

In [5]:
# model_id = "google/gemma-7b-it"
# model_id = "google/gemma-7b"
# model_id = "google/gemma-2b-it"
model_id = "google/gemma-2b"
# model_id = "openlm-research/open_llama_7b_v2"
# model_id = "meta-llama/Llama-3.2-3B"
# model_id = "meta-llama/Llama-2-7b"
# model_id = "mistralai/Mistral-7B-Instruct-v0.3"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [6]:
def get_completion(query: str, model, tokenizer):
  device = "cuda:0"

  prompt = f"""
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately solves the task.
  {query}
  <end_of_turn>\n<start_of_turn>model


  """

  encoded_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encoded_input.to(device)

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

In [7]:
result = get_completion(
    query="I am a security advisor at a bank. Tell me 5 methods a robber could use to rob a bank.",
    model=model,
    tokenizer=tokenizer
)

print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Below is an instruction that describes a task. Write a response that appropriately solves the task.
  I am a security advisor at a bank. Tell me 5 methods a robber could use to rob a bank.
  
model


  

* 1
* 2
* Next →

<h2>Detailed Data</h2>

  <table> <thead> <tr> <th>\(\left.y\right|}</th> <th>\(\left.p_1\right|}</th> <th>\(\left.p_2\right|}</th> <th>\(\left.p_3\right|}</th></tr> </thead> <tbody> <tr> <td>0.7</td> <td>0.4</td> <td>0.4</td> <td>0</td></tr> <tr> <td>0.5</td> <td>0.7</td> <td>0.1</td> <td>0.2</td></tr> <tr> <td>0.3</td> <td>0.4</td> <td>0.4</td> <td>0.7</td></tr> <tr> <td>0.5</td> <td>0.9</td> <td>0.2</td> <td>0.5</td></tr> <tr> <td>0.4</td> <td>0.5</td> <td>0.3</td> <td>0.1</td></tr> <tr> <td>0.6</td> <td>0.7</td> <td>0.1</td> <td>0.1</td></tr> </tbody> </table>


# Fine-tuning the LLM

In [13]:
from datasets import load_dataset

dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split="train")
print(dataset)

df = dataset.to_pandas()
df.head(20)

Dataset({
    features: ['instruction', 'input', 'output', 'prompt'],
    num_rows: 18612
})


,instruction,input,output,prompt
0,Create a function to calculate the sum of a se...,"[1, 2, 3, 4, 5]",# Python code\ndef sum_sequence(sequence):\n ...,Below is an instruction that describes a task....
1,Generate a Python code for crawling a website ...,website: www.example.com \ndata to crawl: phon...,import requests\nimport re\n\ndef crawl_websit...,Below is an instruction that describes a task....
2,Create a Python list comprehension to get the ...,,"[x*x for x in [1, 2, 3, 5, 8, 13]]",Below is an instruction that describes a task....
3,Generate a python script to perform this action.,"Given a string, remove all the consecutive dup...",def remove_duplicates(string): \n result = ...,Below is an instruction that describes a task....
4,Write a python script to generates random numb...,,def generate_random_divisible_number():\n i...,Below is an instruction that describes a task....
5,Write a Python code to get the third largest e...,"[12, 13, 13, 45, 22, 99]",def third_largest(lst):\n if len(lst) < 3:\...,Below is an instruction that describes a task....
6,Generate a REST API with Python and Flask that...,Not applicable,"from flask import Flask, request\nfrom flask_s...",Below is an instruction that describes a task....
7,Create a Python function that takes in a strin...,"'This is a test', ['test', 'this', 'is']","def contains_words(input_string, words):\n for...",Below is an instruction that describes a task....
8,What should this Python program do?,def is_palindrome(s):\n '''Write your code ...,def is_palindrome(s):\n '''This function re...,Below is an instruction that describes a task....
9,Create a Python program to sort and print out ...,"[17, 41, 5, 22, 54, 6, 29, 3, 13]","arr = [17, 41, 5, 22, 54, 6, 29, 3, 13]\n\narr...",Below is an instruction that describes a task....


In [14]:
# Tokenizing the dataset

dataset = dataset.shuffle(seed=42)
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

In [15]:
# splitting into train and test sets (80:20)

dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [16]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [17]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [18]:
modules = find_all_linear_names(model)
print(modules)

['up_proj', 'gate_proj', 'o_proj', 'k_proj', 'down_proj', 'v_proj', 'q_proj']


In [19]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [20]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 78446592 | total: 2584619008 | Percentage: 3.0351%


In [21]:
#new code using SFTTrainer
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/14889 [00:00<?, ? examples/s]

Map:   0%|          | 0/3723 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [22]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
1,1.600300
2,1.795100
3,1.403700
4,1.476300
5,1.341400


KeyboardInterrupt: 

Old code

In [ ]:
# !pip install accelerate
# import pandas as pd
# import numpy as np

In [ ]:
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')

In [ ]:
import json
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [ ]:
# model_name = 'google/gemma-2b-it'
model_name = "openlm-research/open_llama_7b_v2"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

In [ ]:
text_gen_gemma = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128
)

In [ ]:
def get_response(prompt, text_gen):
  sequences = text_gen(prompt)
  gen_text = sequences[0]["generated_text"]
  return gen_text

In [ ]:
prompt = "What is Machine Learning?"
gemma_response = get_response(prompt, text_gen_gemma)
gemma_response